In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris

In [2]:
iris = load_iris()
df = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                     columns= iris['feature_names'] + ['target'])

In [3]:
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0


In [4]:
from sqlalchemy import create_engine

engine = create_engine('sqlite:///irysy.db')

In [5]:
df.to_sql('dane', con=engine, index=False)

In [6]:
a = engine.execute("SELECT * FROM dane").fetchall()

In [7]:
pd.DataFrame(a)

,0,1,2,3,4
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2.0
146,6.3,2.5,5.0,1.9,2.0
147,6.5,3.0,5.2,2.0,2.0
148,6.2,3.4,5.4,2.3,2.0


In [8]:
df2 = pd.DataFrame({'name' : ['User 1', 'User 2', 'User 3']})
df2

,name
0,User 1
1,User 2
2,User 3


In [9]:
df2.to_sql('users', con=engine)
engine.execute("SELECT * FROM users").fetchall()

[(0, 'User 1'), (1, 'User 2'), (2, 'User 3')]

In [10]:
df3 = pd.DataFrame({'name' : ['User 6', 'User 7']})
df3.to_sql('users', con=engine, if_exists='append')

In [11]:
engine.execute("SELECT * FROM users").fetchall()

[(0, 'User 1'), (1, 'User 2'), (2, 'User 3'), (0, 'User 6'), (1, 'User 7')]

In [12]:
from sqlalchemy import create_engine
from sqlalchemy import Column, String, Integer
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

In [13]:
engine = create_engine('sqlite:///irysy.db')
base = declarative_base()

In [14]:
class Transakcje(base):
    
    __tablename__ = 'transakcje'
    
    transakcja_id = Column(Integer, primary_key=True )
    data = Column(String)
    przedmiot_id = Column(Integer)
    cena = Column(Integer)
    
    def __init__(self, transakcja_id, data, przedmiot_id, cena):
        self.transakcja_id = transakcja_id
        self.data = data
        self.przedmiot_id = przedmiot_id
        self.cena = cena

In [15]:
# tworzenie tabel
base.metadata.create_all(engine)

In [16]:
from sqlalchemy.orm import sessionmaker
# Stworzenie nowej sesji
Session = sessionmaker(bind=engine)
session = Session()
# dodanie danych
for t in range(10):
    tr = Transakcje(t, f'200{t}/05/06', t**2-t*2, 19)
    session.add(tr)
# zapis zmian w bazie danych
session.commit()

In [17]:
# Session = sessionmaker(bind=engine)
# session = Session()
# wszystkie dane
for s in session.query(Transakcje).all():
    print(s.transakcja_id, s.data)

0 2000/05/06
1 2001/05/06
2 2002/05/06
3 2003/05/06
4 2004/05/06
5 2005/05/06
6 2006/05/06
7 2007/05/06
8 2008/05/06
9 2009/05/06


In [18]:
# wybrane transakcje
for s in session.query(Transakcje).filter(Transakcje.transakcja_id>5):
    print(s.transakcja_id, s.data)

6 2006/05/06
7 2007/05/06
8 2008/05/06
9 2009/05/06


In [19]:
import json

In [20]:
person = '{"name": "Bob", "languages": ["English", "Fench"]}'

type(person)

str

In [21]:
person_dict = json.loads(person)

# Output: {'name': 'Bob', 'languages': ['English', 'Fench']}
print( person_dict)

# Output: ['English', 'French']
print(person_dict['languages'])

{'name': 'Bob', 'languages': ['English', 'Fench']}
['English', 'Fench']


In [22]:
%%file test.json


{"name": "Bob", 
"languages": ["English", "Fench"]
}

Writing test.json


In [23]:
with open('test.json') as f:
    data = json.load(f)

# Output: {'name': 'Bob', 'languages': ['English', 'Fench']}
print(data)

{'name': 'Bob', 'languages': ['English', 'Fench']}


In [24]:
person_dict = {'name': 'Bob',
'age': 12,
'children': None
}
person_json = json.dumps(person_dict)

print(person_json)


{"name": "Bob", "age": 12, "children": null}


In [25]:
with open('person.json', 'w') as json_file:
    json.dump(person_dict, json_file)

In [26]:
print(json.dumps(person_dict, indent = 4, sort_keys=True))

{
    "age": 12,
    "children": null,
    "name": "Bob"
}


In [29]:
from flask import Flask, request
from flask_restful import Resource, Api

from flask_jsonpify import jsonify

In [30]:
app  = Flask(__name__)
api = Api(app)

@app.route('/')
@app.route('/index')
def home():
    #return render_template('home.html')
    return "Strona poczatkowa"

@app.route('/hello/<name>')
def success(name):
    return f'<h1>{name}</h1>'

In [31]:
class Main(Resource):
    def get(self):
        return jsonify("Hello world")

In [32]:
api.add_resource(Main,'/test')

In [33]:
class Irys(Resource):
    
    def get(self):
        conn=engine.connect()
        query = conn.execute('select * from dane')
        result = {'dane':[i for i in query.cursor.fetchall()]}
        return jsonify(result)

In [34]:
api.add_resource(Irys, '/irys')

In [35]:
app.run(port='5002')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5002/ (Press CTRL+C to quit)
127.0.0.1 - - [15/Apr/2021 19:37:01] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2021 19:37:01] "GET /favicon.ico HTTP/1.1" 404 -
